In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import keras
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
df = pd.read_csv('Bama.csv')

In [3]:
df.head()

,Unnamed: 0,brand,model,submodel,year,location,price,gearbox,operations,body,gas,color,insidecolor
0,0,پژو,207,پانوراما دنده ای,1402,تهران,NaN,0,0.0,بدون رنگ,بنزینی,مشکی,داخل مشکی
1,1,سمند,سورن,پلاس بنزینی,1401,قم,538.0,0,28000.0,بدون رنگ,بنزینی,سفید,داخل مشکی
2,2,پژو,206,تیپ 2,1399,تهران,430.0,0,30000.0,بدون رنگ,بنزینی,خاکستری,داخل مشکی
3,3,پژو,پارس,XU7P,1401,تهران,488.0,0,13000.0,بدون رنگ,بنزینی,مشکی,داخل مشکی
4,4,پژو,2008,اتوماتیک,1397,کرمان,NaN,1,12000.0,بدون رنگ,بنزینی,سفید,داخل مشکی


In [4]:
df.drop('Unnamed: 0',axis=1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3926 entries, 0 to 3925
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   brand        3926 non-null   object 
 1   model        3926 non-null   object 
 2   submodel     3926 non-null   object 
 3   year         3926 non-null   int64  
 4   location     3926 non-null   object 
 5   price        2463 non-null   float64
 6   gearbox      3926 non-null   int64  
 7   operations   3845 non-null   float64
 8   body         3926 non-null   object 
 9   gas          3926 non-null   object 
 10  color        3926 non-null   object 
 11  insidecolor  3926 non-null   object 
dtypes: float64(2), int64(2), object(8)
memory usage: 368.2+ KB


In [6]:
df.describe()

,year,price,gearbox,operations
count,3926.000000,2463.000000,3926.000000,3845.000000
mean,1395.118696,1380.240784,0.616913,99094.014564
std,5.974866,1840.283478,0.486201,100549.368628
min,1349.000000,0.850000,0.000000,0.000000
25%,1392.000000,406.000000,0.000000,18000.000000
50%,1396.000000,720.000000,1.000000,78700.000000
75%,1400.000000,1580.000000,1.000000,150000.000000
max,1402.000000,15000.000000,1.000000,990000.000000


In [7]:
df.isnull().sum()

brand             0
model             0
submodel          0
year              0
location          0
price          1463
gearbox           0
operations       81
body              0
gas               0
color             0
insidecolor       0
dtype: int64

In [8]:
df.head()

,brand,model,submodel,year,location,price,gearbox,operations,body,gas,color,insidecolor
0,پژو,207,پانوراما دنده ای,1402,تهران,NaN,0,0.0,بدون رنگ,بنزینی,مشکی,داخل مشکی
1,سمند,سورن,پلاس بنزینی,1401,قم,538.0,0,28000.0,بدون رنگ,بنزینی,سفید,داخل مشکی
2,پژو,206,تیپ 2,1399,تهران,430.0,0,30000.0,بدون رنگ,بنزینی,خاکستری,داخل مشکی
3,پژو,پارس,XU7P,1401,تهران,488.0,0,13000.0,بدون رنگ,بنزینی,مشکی,داخل مشکی
4,پژو,2008,اتوماتیک,1397,کرمان,NaN,1,12000.0,بدون رنگ,بنزینی,سفید,داخل مشکی


In [9]:
label = LabelEncoder()
df['model'] = label.fit_transform(df['model'])
df['submodel'] = label.fit_transform(df['submodel'])
df['location'] = label.fit_transform(df['location'])
# df['gas'] = label.fit_transform(df['gas'])
df['color'] = label.fit_transform(df['color'])
df['insidecolor'] = label.fit_transform(df['insidecolor'])

df = pd.get_dummies(df,columns=['brand','body','gas'])

df['price'].fillna(round(df['price'].mean(), 0),inplace=True)
df['operations'].fillna(round(df['operations'].mean(), 0),inplace=True)

C:\Users\Mahdi\AppData\Local\Temp\ipykernel_21440\1150894517.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(round(df['price'].mean(), 0),inplace=True)
C:\Users\Mahdi\AppData\Local\Temp\ipykernel_21440\1150894517.py:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a

In [10]:
X = df.drop('price',axis=1)
y = df['price']

X_train_valid, X_test, y_train_valid, y_test = train_test_split(X,y,test_size=0.2,random_state=39)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid,y_train_valid,test_size=0.2,random_state=39)

In [11]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [12]:
def create_model(n_hidden=1,n_neuron=64,input_shape=(X_train.shape[1],)):
    model = keras.Sequential()
    options = {'input_shape':input_shape}
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neuron,activation='relu',**options))
        model.add(keras.layers.Dropout(0.2))
        options = {}
    model.add(keras.layers.Dense(1,))
    model.compile(optimizer='adam',loss='mse')
    return model

In [13]:
model_chack_point = ModelCheckpoint(filepath='best_model.keras',save_best_only=True)
early = EarlyStopping(patience=10,restore_best_weights=True)

In [14]:
model = KerasRegressor(model=create_model,verbose=0)

In [15]:
params_randomized = {'model__n_hidden':[1,2,3,4],'model__n_neuron':[128,64,32]}

randomized = RandomizedSearchCV(model,params_randomized,scoring='neg_mean_squared_log_error',n_iter=10,cv=3)
hist = randomized.fit(X_train,y_train,epochs=100,batch_size=16,validation_data=(X_valid,y_valid),callbacks=[model_chack_point,early],verbose=0)

c:\Users\Mahdi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Mahdi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Mahdi\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in

In [17]:
finall_model = randomized.best_estimator_
y_pred = finall_model.predict(X_test)
mse = mean_squared_error(y_test,y_pred)
rmse = round(np.sqrt(mse), 2)
score = round(finall_model.score(X_test,y_test) * 100, 2)
print('RMSE: ',rmse)
print('Score: ',score)

RMSE:  1362.51
Score:  29.13
